In [24]:
import requests 
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import jieba #中文斷詞函式庫

headers = requests.utils.default_headers()
headers['User-Agent'] = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'
#解決有些網站無法被爬蟲的問題

In [ ]:
driver = webdriver.Chrome('/Users/richard/Documents/0Python/chromedriver')
#載入webdriver

wait = ui.WebDriverWait(driver,60)
#載入等候時間（60秒內持續掃瞄）

In [ ]:
keyword = '水果'
url = 'https://find.ruten.com.tw/s/?q={}'.format(keyword) 

driver.get(url) #用瀏覽器開啟網頁
soup = BeautifulSoup(driver.page_source, 'html.parser') #用bs解析網頁

In [ ]:
max_page = int(soup.find('li', class_='info').text[1:])  #用find找到頁數，用index排除數字１，找到最大頁數，再用int化為數字 #決定後續翻頁次數使用

max_page

In [ ]:
nameL = []
for i in range(1, max_page+1):
    new_url = url+'&p='+str(i) 
    driver.get(new_url) 
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    original_nameL = soup.select('h5>a') #抓符合條件的關鍵字（標題）
    for i in original_nameL:
        name = i.get_text()
        seg_list = jieba.lcut(name, cut_all=False) #抓出關鍵字後就先斷詞
        nameL += seg_list

In [ ]:
stopWords=[]
remainderWords=[]

with open('/Users/richard/Documents/0Python/WebCrawling/stopWords.txt', 'r', encoding='UTF-8') as file:
    for data in file.readlines():
        data = data.strip()
        stopWords.append(data) #建立廢詞庫

In [ ]:
remainderWords = list(filter(lambda a: a not in stopWords and a != '\n', nameL)) #用斷詞後結果與廢詞比對，留下非廢詞

In [ ]:
dic = {}

for i in remainderWords:
    if i.title() in dic:
        dic[i.title()] += 1 #計算出現次數
    else:
        dic[i.title()] = 1
        

df = pd.DataFrame(list(dic.items()), columns=['關鍵字', '出現次數'])

df_sort = df.sort_values(by=['出現次數'], ascending = False).reset_index(drop = True)
df_sort['搜尋結果'] = 'https://find.ruten.com.tw/s/?q='+df_sort['關鍵字'] #新增搜尋結果，方便點擊連結
df_20 = df_sort.head(20)
df_20

In [ ]:
df_sort.to_excel('露天_'+keyword+'_全關鍵字.xlsx') #輸出檔案